In [12]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
lgcg_path = os.path.abspath(os.path.join('../lgcg'))
if lgcg_path not in sys.path:
    sys.path.append(lgcg_path)
from lib.measure import Measure

# Generate Data

In [2]:
Omega = np.array([[0,1],[0,1]])
observation_resolution = 5
observations = (np.array(np.meshgrid(
                    *(
                        np.linspace(bound[0], bound[1], observation_resolution)
                        for bound in Omega
                    ))
            ).reshape(len(Omega), -1).T)
std_factor = 0.5
k = lambda x: np.array([np.exp(-np.linalg.norm(point-x)**2/std_factor)/np.sqrt(std_factor*np.pi)**Omega.shape[0] for point in observations])
true_sources = np.array([[0.1,0.3], [0.25,0.8], [0.7,0.5]])
true_weights = np.array([5,-3,7])
target = sum([weight*k(source) for weight, source in zip(true_weights, true_sources)])
norm_K_star = np.sqrt(len(observations)/(std_factor*np.pi))
norm_K_star_L = 2*np.sqrt(len(observations)*Omega.shape[0]/(np.pi*std_factor**3))/std_factor

In [13]:
def p_raw(u, k, target):
    def p_u(x):
        Ku = sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        inner = Ku-target
        return -np.dot(inner, k(x))
    return p_u

p = lambda u: p_raw(u, k=k, target=target)

In [20]:
def grad_P_raw(k, target, observations, std_factor):
    def grad_P(u, x):
        def p_u(x):
            Ku = sum(k(x)*c for c, x in zip(u.coefficients, u.support))
            inner = Ku-target
            return -np.dot(inner, k(x))
        Ku = sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        inner = Ku-target
        second_inner = k(x)*2/std_factor
        vector_part = sum([inr*second_inr*(obs-x) for inr, second_inr, obs in zip(inner, second_inner, observations)])
        return -np.sign(p_u(x))*vector_part
    return grad_P

grad_P = grad_P_raw(k, target, observations, std_factor)

In [3]:
def grad_k_raw(observations, std_factor):
    def grad_k(x):
        return np.array([2*(point-x)*np.exp(-np.linalg.norm(point-x)**2/std_factor)/(std_factor*np.sqrt(std_factor*np.pi)**Omega.shape[0]) for point in observations])
    return grad_k

grad_k = grad_k_raw(observations, std_factor) # The Jacobian of k, shape=(len(observations), Omega.shape[0])

In [8]:
def hess_k_raw(observations, std_factor):
    def hess_k(x):
        first_part = lambda point: -2*np.eye(Omega.shape[0])*np.exp(-np.linalg.norm(point-x)**2/std_factor)/(std_factor*np.sqrt(std_factor*np.pi)**Omega.shape[0])
        second_part = lambda point: 4*np.dot((point-x).reshape(-1,1), np.array([point-x]))*np.exp(-np.linalg.norm(point-x)**2/std_factor)/(std_factor**2*np.sqrt(std_factor*np.pi)**Omega.shape[0])
        return np.array([first_part(point)+second_part(point) for point in observations])
    return hess_k

hess_k = hess_k_raw(observations, std_factor) # The derivative of the Jacobian of k, shape=(len(observations), Omega.shape[0], Omega.shape[0])

In [32]:
def grad_P_raw(k, target, observations, std_factor):
    grad_k = grad_k_raw(observations, std_factor)
    def grad_P(x,u):
        p_u = p_raw(u, k, target)
        inner = target-sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        return np.sign(p_u(x))*np.matmul(grad_k(x).T, inner)
    return grad_P

grad_P = grad_P_raw(k, target, observations, std_factor)

In [33]:
def hess_P_raw(k, target, observations, std_factor):
    hess_k = hess_k_raw(observations, std_factor)
    def hess_P(x,u):
        p_u = p_raw(u, k, target)
        inner = target-sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        return np.sign(p_u(x))*np.tensordot(hess_k(x),inner,axes=([0,0]))
    return hess_P

hess_P = hess_P_raw(k, target, observations, std_factor)

In [30]:
target

array([3.15191494, 3.81351948, 3.95712709, 3.50943456, 2.61142507,
       3.66011727, 4.6079549 , 5.00984175, 4.64102819, 3.57307225,
       3.14489828, 4.18610067, 4.83961186, 4.72874385, 3.78584505,
       1.87996289, 2.75232252, 3.49495837, 3.67245274, 3.08807258,
       0.62956544, 1.18183781, 1.80673047, 2.13326966, 1.92170735])

In [22]:
observations

array([[0.  , 0.  ],
       [0.25, 0.  ],
       [0.5 , 0.  ],
       [0.75, 0.  ],
       [1.  , 0.  ],
       [0.  , 0.25],
       [0.25, 0.25],
       [0.5 , 0.25],
       [0.75, 0.25],
       [1.  , 0.25],
       [0.  , 0.5 ],
       [0.25, 0.5 ],
       [0.5 , 0.5 ],
       [0.75, 0.5 ],
       [1.  , 0.5 ],
       [0.  , 0.75],
       [0.25, 0.75],
       [0.5 , 0.75],
       [0.75, 0.75],
       [1.  , 0.75],
       [0.  , 1.  ],
       [0.25, 1.  ],
       [0.5 , 1.  ],
       [0.75, 1.  ],
       [1.  , 1.  ]])

In [34]:
grad_P(np.array([0.1,0.1]), Measure())

array([26.73401091, 20.74271496])

In [35]:
hess_P(np.array([0.1,0.1]), Measure())

array([[-38.93010246,  24.30044838],
       [ 24.30044838, -56.66942886]])

# Test Stochastic

In [2]:
from lgcg_stochastic import LGCG_stochastic

In [3]:
data_path = "../../sisso/src/test/thermal_conductivity_data.csv"
exp = LGCG_stochastic(data_path=data_path, alpha=0.1)

In [4]:
exp.solve(tol=1e-7)

INFO:root:---------------------
INFO:root:---------------------


Time to generate feat space: 262.406 s


INFO:root:0.9755782288434427
INFO:root:1
INFO:root:K []
INFO:root:[0.01650826 0.08398484 0.03875523 0.03877598 0.04318637 0.04194306
 0.04804311 0.06922419 0.08354961 0.11852333 0.04570393 0.04838697
 0.06082152 0.04974988 0.03391667 0.03654056 0.03532187 0.03515929
 0.04248941 0.04185097 0.0362112  0.07187031 0.0379061  0.04252391
 0.03777911 0.13475375 0.12981585 0.11154833 0.34266939 0.21564257
 0.06654163 0.12626913 0.13974483 0.09623659 0.11400463 0.09425901
 0.15303787 0.08798618 0.07927755 0.18412779 0.18997568 0.23295921
 0.08915301 0.20109564 0.13541286 0.09429843 0.07918945 0.06793185
 0.06641488 0.1338539  0.12592129 0.12139781 0.13090409 0.1034475
 0.08834254 0.1011218  0.05419033 0.06345643 0.05942181 0.05300143
 0.0721522  0.09091953 0.08603043 0.10407362 0.1014441  0.065165
 0.19817434 0.27812532 0.24831032 0.06895994 0.13319063 0.11928598
 0.08974987 0.10409334 0.06904784]
/vol/cs-hu/hnatiuar@hu-berlin.de/Dokumente/lazified-generalized-conditional-gradient/lgcg/lgcg_sto

Time to generate feat space: 262.994 s


INFO:root:0.9539958314747843
INFO:root:1
INFO:root:['((omega_Gamma_max / sigma_os) / (Theta_p * L_min_prim))']
INFO:root:2: Phi 3.106E+00, epsilon 6.361E+00, support 1, Psi 5.050E+00
INFO:root:K [[0.01650826 0.08398484 0.03875523 0.03877598 0.04318637 0.04194306
  0.04804311 0.06922419 0.08354961 0.11852333 0.04570393 0.04838697
  0.06082152 0.04974988 0.03391667 0.03654056 0.03532187 0.03515929
  0.04248941 0.04185097 0.0362112  0.07187031 0.0379061  0.04252391
  0.03777911 0.13475375 0.12981585 0.11154833 0.34266939 0.21564257
  0.06654163 0.12626913 0.13974483 0.09623659 0.11400463 0.09425901
  0.15303787 0.08798618 0.07927755 0.18412779 0.18997568 0.23295921
  0.08915301 0.20109564 0.13541286 0.09429843 0.07918945 0.06793185
  0.06641488 0.1338539  0.12592129 0.12139781 0.13090409 0.1034475
  0.08834254 0.1011218  0.05419033 0.06345643 0.05942181 0.05300143
  0.0721522  0.09091953 0.08603043 0.10407362 0.1014441  0.065165
  0.19817434 0.27812532 0.24831032 0.06895994 0.13319063 0.1